In [1]:
import numpy as np
import codecs
import tensorflow as tf
import pandas as pd
import ast
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
import joblib
train = joblib.load("/content/drive/MyDrive/mmm/train")
validation = joblib.load("/content/drive/MyDrive/mmm/validation")
train.head(5)

,question,answer_in,answer_out
764408,you need to work on your advertising all video...,<start> we apologize for the frustration this ...,we apologize for the frustration this might be...
1991,cable amp internet out all day called cs x is ...,<start> sorry our tech never came i can check ...,sorry our tech never came i can check the acco...
857358,i think i am done with they are really playing...,<start> we can definitely take a look send us ...,we can definitely take a look send us a dm wit...
15701,will hand this book in at paddington found on ...,<start> hi cameron thanks for confirming i wil...,hi cameron thanks for confirming i will re twe...
626333,tried to book a room money deducted shows fail...,<start> hi there we are sorry to hear about th...,hi there we are sorry to hear about this can y...


In [3]:
tknizer_q = Tokenizer(filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_q.fit_on_texts(train['question'].values)
tknizer_a = Tokenizer(filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_a.fit_on_texts(train['answer_in'].values)

In [4]:
q_word_idx = tknizer_q.word_index
q_idx_word = {v: k for k, v in q_word_idx.items()}

a_word_idx = tknizer_a.word_index
a_idx_word = {v: k for k, v in a_word_idx.items()}

print(len(q_word_idx.keys()))
print(len(a_word_idx.keys()))

139809
57128


In [5]:
train['answer_in'].values


array(['<start> we apologize for the frustration this might be causing we appreciate your feedback and we will share it with our advertising teams <end>',
       '<start> sorry our tech never came i can check the account if you could dm your address or acct ps',
       '<start> we can definitely take a look send us a dm with your email address so we can follow up',
       ...,
       '<start> hi stephen we want to follow up can you please dm us the guests information so that we can reach out to her and review the case',
       '<start> hello kirsty i am very sorry to hear you have not received your early access code can you dm us your full name email address and full home address details which are linked to your account please so we can look into this please ty ross',
       '<start> that is not the experience we would hope you have send us a dm and let us know more details we will get started there'],
      dtype=object)

In [6]:
a_word_idx['<start>'], a_word_idx['<end>']


(1, 30128)

In [7]:
vocab_size_a = len(a_word_idx.keys())
vocab_size_q = len(q_word_idx.keys())

In [8]:
#This class represents a custom dataset for the sequence-to-sequence model.
#It takes data (presumably a DataFrame), along with tokenizers (tknizer_q and tknizer_a) for questions and answers.
class Dataset:
    def __init__(self, data, tknizer_q, tknizer_a):
        self.encoder_inps = data['question'].values
        self.decoder_inps = data['answer_in'].values
        self.decoder_outs = data['answer_out'].values
        self.tknizer_q = tknizer_q
        self.tknizer_a = tknizer_a

#The __getitem__ method converts text sequences to sequences of integers using tokenizers and pads them to a fixed length (27 in this case).
    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_q.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_a.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_a.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq,maxlen=27, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq,maxlen=27, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq,maxlen=27, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

#The __len__ method returns the total number of samples in the dataset.
    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)



#This class is a custom data loader inheriting from tf.keras.utils.Sequence, which is suitable for generating data batches in parallel while training.
#It takes a dataset (an instance of the Dataset class) and a batch_size as parameters.
class Dataloder(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))

#The __getitem__ method retrieves a batch of data from the dataset, stacks the samples, and returns a tuple of input and output sequences.
    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        return tuple([[batch[0],batch[1]],batch[2]])

#The __len__ method returns the total number of batches in an epoch.

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size
#The on_epoch_end method shuffles the indexes at the end of each epoch.
    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [9]:
train_dataset = Dataset(train, tknizer_q, tknizer_a)
test_dataset  = Dataset(validation, tknizer_q, tknizer_a)
train_dataloader = Dataloder(train_dataset, batch_size=128)
test_dataloader = Dataloder(test_dataset, batch_size=128)

print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(128, 27) (128, 27) (128, 27)


In [10]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''
    def __init__(self, ques_vocab_size, embedding_dim, encoder_input_length, lstm_units):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__(name="encode_model_attention")
        self.ques_vocab_size = ques_vocab_size
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.encoder_input_length = encoder_input_length
        self.embedding = Embedding(input_dim=self.ques_vocab_size, output_dim=self.embedding_dim, input_length=self.encoder_input_length,\
                  mask_zero=True, name="embedding_layer_encoder")
        self.LSTM = LSTM(self.lstm_units, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
        '''
        input_embeddings = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h, self.lstm_state_c = self.LSTM(input_embeddings)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c


    def initialize_states(self,batch_size):
        '''
        Given a batch size it will return intial hidden state and intial cell state.
        If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
        '''
        lstm_state_h = tf.zeros([batch_size, self.lstm_size])
        lstm_state_c = tf.zeros([batch_size, self.lstm_size])

        return lstm_state_h, lstm_state_c

In [11]:
from tensorflow.keras.layers import *
class Attention(tf.keras.layers.Layer):
    '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
    '''
    def __init__(self, att_units):

        super().__init__(name="Attention_Bahdanau")
        self.K = 100
        self.dense_1 = Dense(self.K, activation='relu')
        self.dense_2 = Dense(self.K, activation='relu')
        self.dense_3 = Dense(1, activation='relu')


    def call(self, decoder_hidden_state, encoder_output):
        '''
          Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
          * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
            Multiply the score function with your encoder_outputs to get the context vector.
            Function returns context vector and attention weights(softmax - scores)
        '''

        k1 = self.dense_1(encoder_output)
        k2 =  self.dense_2(decoder_hidden_state)
        add = tf.keras.layers.Add()([k1, k2])
        tanh = tf.keras.layers.Activation(activation="tanh")(add)
        ei = self.dense_3(tanh)
        ei = tf.squeeze(ei,-1)
        alphas = Softmax()(ei)
        alphas = tf.expand_dims(alphas, axis=-1)
        mull = tf.keras.layers.Multiply()([encoder_output, alphas])
        context_vec = tf.reduce_mean(mull, axis=-2)
        return context_vec, alphas

In [12]:
class OneStepDecoder(tf.keras.Model):
    def __init__(self,ans_vocab_size, embedding_dim, decoder_input_length, lstm_units, attention_units):
        super().__init__(name="OneStepDecoder")
        # Initialize decoder embedding layer, LSTM and any other objects needed

        self.lstm_units = lstm_units
        self.attention_units = attention_units
        self.attention = Attention(attention_units)
        self.embedding = Embedding(input_dim = ans_vocab_size, output_dim = embedding_dim, input_length = decoder_input_length,
                                   mask_zero=True, trainable=True )
        #weights=[embedding_matrix]

        self.lstm = LSTM(self.lstm_units, return_sequences=True, return_state=True,name="OneStepDecoder_LSTM")
        self.dense = Dense(ans_vocab_size, activation='softmax')


    def call(self, input_to_decoder, encoder_output, state_h, state_c):
        '''
             One step decoder mechanisim step by step:
          A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
          B. Using the encoder_output and decoder hidden state, compute the context vector.
          C. Concat the context vector with the step A output
          D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
          E. Pass the decoder output to dense layer(vocab size) and store the result into output.
          F. Return the states from step D, output from Step E, attention weights from Step -B
        '''
        #adding attention information to embeddings now
        embeddings = self.embedding(input_to_decoder)
        context_vector, attention_weights = self.attention(state_h, encoder_output)
        attention_embeddings = tf.concat([tf.expand_dims(context_vector, 1), embeddings], axis=-1)

        x, state_h, state_c = self.lstm(attention_embeddings, initial_state=[state_h, state_c])

        output = self.dense(x)

        output = tf.squeeze(output,1)

        return output, state_h, state_c, attention_weights, context_vector

In [13]:
class Decoder(tf.keras.Model):
    def __init__(self, ans_vocab_size, embedding_dim, decoder_input_length, lstm_units, attention_units):
        #Intialize necessary variables and create an object from the class onestepdecoder
        super().__init__(name="Decode_Attention")
        self.ans_vocab_size = ans_vocab_size
        self.embedding_dim = embedding_dim
        self.decoder_input_length = decoder_input_length
        self.lstm_units = lstm_units
        self.attention_units = attention_units
        self.onestepdecoder = OneStepDecoder(self.ans_vocab_size, \
                                             self.embedding_dim, \
                                             self.decoder_input_length, \
                                             self.lstm_units, self.attention_units)

    def call(self, input_to_decoder, encoder_output, decoder_hidden_state, decoder_cell_state):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook

        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array

        all_outputs = tf.TensorArray(tf.float32, size=27, name="output_arrays")
        for timestep in range(27):
            output,decoder_hidden_state,decoder_cell_state,attention_weights,context_vector = self.onestepdecoder(\
                                input_to_decoder[:, timestep:timestep+1],encoder_output,\
                                decoder_hidden_state,decoder_cell_state
                               )

            all_outputs = all_outputs.write(timestep, output)
        all_outputs = tf.transpose(all_outputs.stack(), [1,0,2])
        return all_outputs

In [14]:
class encoder_decoder(tf.keras.Model):
    def __init__(self,ques_vocab_size, ans_vocab_size, encoder_input_length, dencoder_input_length, emb_dim, lstm_units, attention_units):
        #Intialize objects from encoder decoder
        super().__init__(name="Encoder_Decoder_Attention_model")

        self.encoder = Encoder(ques_vocab_size, emb_dim, encoder_input_length, lstm_units )
        self.decoder = Decoder(ans_vocab_size, emb_dim, dencoder_input_length, lstm_units, attention_units)

    def call(self,data):
        #Intialize encoder states, Pass the encoder_sequence to the embedding layer
        # Decoder initial states are encoder final states, Initialize it accordingly
        # Pass the decoder sequence,encoder_output,decoder states to Decoder
        # return the decoder output
        input_q, input_a = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input_q)
        decoder_output = self.decoder(input_a, encoder_output, encoder_h, encoder_c)
        return decoder_output

In [15]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow.keras.utils import to_categorical


from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, TensorBoard, LearningRateScheduler
from sklearn.metrics import recall_score, f1_score, roc_curve, auc
import datetime

filepath="model_save_attention_4/weights-{epoch:02d}-{val_loss:.2f}"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', save_format="tf", save_freq="epoch",  verbose=1, save_best_only=True, mode='auto')


In [ ]:
ques_vocab_size = vocab_size_q + 1
ans_vocab_size = vocab_size_a + 1
encoder_input_length = 27
dencoder_input_length = 27
lstm_units = 512
attention_units = 512
emb_dim = 100

model_attention = encoder_decoder(ques_vocab_size,
                                  ans_vocab_size,
                                  encoder_input_length,
                                  dencoder_input_length,
                                  emb_dim,
                                  lstm_units,
                                  attention_units)

optimizer = tf.keras.optimizers.Adam()
model_attention.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


train_steps=train.shape[0]//128
valid_steps=validation.shape[0]//128
history_dot = model_attention.fit(train_dataloader, \
                                      steps_per_epoch=train_steps, \
                                      epochs=10, \
                                      validation_data=test_dataloader, \
                                      validation_steps=valid_steps,
                                      callbacks=[checkpoint])

Epoch 1/10
4964/4964 [==============================] - ETA: 0s - loss: 2.8514 - accuracy: 0.5082
Epoch 1: val_loss improved from inf to 2.13657, saving model to model_save_attention_4/weights-01-2.14


4964/4964 [==============================] - 3645s 723ms/step - loss: 2.8514 - accuracy: 0.5082 - val_loss: 2.1366 - val_accuracy: 0.5906
Epoch 2/10
4964/4964 [==============================] - ETA: 0s - loss: 1.9995 - accuracy: 0.6106
Epoch 2: val_loss improved from 2.13657 to 1.90270, saving model to model_save_attention_4/weights-02-1.90


4964/4964 [==============================] - 3610s 727ms/step - loss: 1.9995 - accuracy: 0.6106 - val_loss: 1.9027 - val_accuracy: 0.6247
Epoch 3/10
4964/4964 [==============================] - ETA: 0s - loss: 1.8191 - accuracy: 0.6351
Epoch 3: val_loss improved from 1.90270 to 1.80878, saving model to model_save_attention_4/weights-03-1.81


4964/4964 [==============================] - 3622s 730ms/step - loss: 1.8191 - accuracy: 0.6351 - val_loss: 1.8088 - val_accuracy: 0.6391
Epoch 4/10
 596/4964 [==>...........................] - ETA: 47:38 - loss: 1.7217 - accuracy: 0.6476

In [ ]:
from keras.utils import plot_model
plot_model(model_attention)